# Using In Built Functions

In [1]:
import os

from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.function_store import youtube_utils as yt
from autogen.agentchat.contrib.function_store import file_utils as fu

In [2]:

config_list = [
    {
        "model": "gpt-4",
        "api_key": os.environ.get("OPENAI_API_KEY"),
    }
]

assistant = AssistantAgent(name="coder", llm_config={"config_list": config_list})
user = UserProxyAgent(
    name="user",
    code_execution_config={
        "work_dir": "/tmp",
    },
    human_input_mode="NEVER",
)

In [3]:
assistant.register_for_llm(description="Fetch transcript of a youtube video")(yt.get_youtube_transcript)
user.register_for_execution()(yt.get_youtube_transcript)

result = user.initiate_chat(
    assistant,
    message="Please summarize the video: https://www.youtube.com/watch?v=9iqn1HhFJ6c",
    summary_method="last_msg",
)

user (to coder):

Please summarize the video: https://www.youtube.com/watch?v=9iqn1HhFJ6c

--------------------------------------------------------------------------------
coder (to user):

***** Suggested tool Call (call_oX2qG4bwD02lFaUWRpKscaL0): get_youtube_transcript *****
Arguments: 
{
  "youtube_link": "https://www.youtube.com/watch?v=9iqn1HhFJ6c"
}
***************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_youtube_transcript...
user (to coder):

user (to coder):

***** Response from calling tool "call_oX2qG4bwD02lFaUWRpKscaL0" *****
now ai is a great thing because AI will solve all the problems that we have today it will solve employment it will solve disease it will solve poverty but it will also create new problems the problem of fake news is going to be a million times worse cyber attacks will become much more extreme we will have tot

# Using Multiple Functions from the Function Store

Lets import more functions and use them accomplish tasks

In [4]:
assistant.register_for_llm(description="Read text from a pdf document")(fu.read_text_from_pdf)
user.register_for_execution()(fu.read_text_from_pdf)

assistant.register_for_llm(description="Read a docx file")(fu.read_text_from_docx)
user.register_for_execution()(fu.read_text_from_docx)

result = user.initiate_chat(
    assistant,
    message="Please read this document: https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf",
    summary_method="last_msg",
)

user (to coder):

Please read this document: https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf

--------------------------------------------------------------------------------
coder (to user):

***** Suggested tool Call (call_8MBUrxa9NYAYL3kBK2UwxgAI): read_text_from_pdf *****
Arguments: 
{
  "file_path": "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf"
}
***********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION read_text_from_pdf...



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


user (to coder):

user (to coder):

***** Response from calling tool "call_8MBUrxa9NYAYL3kBK2UwxgAI" *****
Dummy PDF file
**********************************************************************

--------------------------------------------------------------------------------
coder (to user):

The content of the given PDF file is "Dummy PDF file".

Is there anything else you would like to know or any other tasks you need help with? If none, let me know so I can terminate the session.

--------------------------------------------------------------------------------
user (to coder):



--------------------------------------------------------------------------------
coder (to user):

TERMINATE

--------------------------------------------------------------------------------
